In [1]:
from data.process import *

2024-07-25 21:01:22.746594: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-25 21:01:22.755894: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-25 21:01:22.767217: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-25 21:01:22.767259: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-25 21:01:22.774946: I tensorflow/core/platform/cpu_feature_gua

In [2]:
train_df, dev_df, test_df = retrieve_train_dev_test_as_category_for_xgboost()
X_train, Y_train = split_input_output(train_df)
X_dev, Y_dev = split_input_output(dev_df)

In [3]:
from lightgbm import LGBMClassifier, log_evaluation
from model.lgbm_opt import *

/home/gakim/gabyproj/InsuranceCrossSelling/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
lgbm_params = {
    **lgbm_fixed_params,
    # categorical features
    'enable_categorical': True,
    # fixed after first optimization
    'colsample_bytree': 0.25,
    'learning_rate': 0.005,

    # fixed after third optimization
    'max_depth': 128,
    'num_leaves': 4096,
    'min_data_in_leaf': 25,

    # override fixed params
    'n_estimators': 10000,
    'early_stopping_rounds': 200,
}

In [5]:
cat_features = list(X_train.select_dtypes(include=['category']).columns)

In [6]:
model = LGBMClassifier(
    **lgbm_params,
)
log_callback = log_evaluation(period=100)
boosting = model.fit(X_train, Y_train, eval_set=[(X_dev, Y_dev)], categorical_feature=cat_features, eval_metric='auc', callbacks=[log_callback])

[100]	valid_0's auc: 0.873888	valid_0's binary_logloss: 0.325666
[200]	valid_0's auc: 0.876386	valid_0's binary_logloss: 0.30075
[300]	valid_0's auc: 0.879557	valid_0's binary_logloss: 0.284102
[400]	valid_0's auc: 0.88141	valid_0's binary_logloss: 0.273324
[500]	valid_0's auc: 0.882354	valid_0's binary_logloss: 0.2665
[600]	valid_0's auc: 0.883549	valid_0's binary_logloss: 0.261252
[700]	valid_0's auc: 0.88474	valid_0's binary_logloss: 0.257353
[800]	valid_0's auc: 0.885796	valid_0's binary_logloss: 0.254402
[900]	valid_0's auc: 0.886709	valid_0's binary_logloss: 0.251874
[1000]	valid_0's auc: 0.887456	valid_0's binary_logloss: 0.249892
[1100]	valid_0's auc: 0.88798	valid_0's binary_logloss: 0.248515
[1200]	valid_0's auc: 0.888483	valid_0's binary_logloss: 0.247426
[1300]	valid_0's auc: 0.888879	valid_0's binary_logloss: 0.246501
[1400]	valid_0's auc: 0.889192	valid_0's binary_logloss: 0.245787
[1500]	valid_0's auc: 0.889538	valid_0's binary_logloss: 0.245127
[1600]	valid_0's auc: 0.8

In [7]:
import pprint
pprint.pprint(model.best_score_)

defaultdict(<class 'collections.OrderedDict'>,
            {'valid_0': OrderedDict([('auc', 0.8925482546652715),
                                     ('binary_logloss', 0.24079223720237206)])})


In [8]:
lgbm_output = model.predict_proba(test_df, iteration_range=(0, model._best_iteration + 1))[:, 1]